<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/transfer-learning/transfer-learning-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (MNIST DATASET)
---

In [1]:
versioninfo()

Julia Version 1.10.8
Commit 4c16ff44be8 (2025-01-22 10:06 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = 2


In [2]:
using Pkg; Pkg.add(["DataAugmentation", "Flux", "Images", "ProgressMeter", "MLDatasets", "Metalhead", "CUDA", "cuDNN"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
Precompiling packages...
           ✗ libwebp_jll
           ✗ Qt5Base_jll
           ✗ GR_jll
           ✗ WebP
           ✗ Plots
           ✗ Plots → UnitfulExt
           ✗ Plots → IJuliaExt
           ✗ Plots → FileIOExt
           ✗ Plots → GeometryBasicsExt
  0 dependencies successfully precompiled in 57 seconds. 520 already precompiled.
  9 dependencies errored.
  For a report of the errors see `julia> err`. To retry use `pkg> precompile`


In [3]:
Pkg.status()

Status `~/.julia/environments/v1.10/Project.toml`
  [336ed68f] CSV v0.10.15
  [052768ef] CUDA v5.6.1
  [88a5189c] DataAugmentation v0.3.2
  [a93c6f00] DataFrames v1.7.0
  [587475ba] Flux v0.16.3
  [7073ff75] IJulia v1.26.0
  [916415d5] Images v0.26.2
  [eb30cadb] MLDatasets v0.7.18
  [ee78f7c6] Makie v0.22.2
  [dbeba491] Metalhead v0.9.5
⌃ [91a5bcdd] Plots v1.40.7
  [92933f4c] ProgressMeter v1.10.2
  [02a925ec] cuDNN v1.4.1
Info Packages marked with ⌃ have new versions available and may be upgradable.


Load the pre-trained model

In [4]:
using Metalhead

[API Reference](https://fluxml.ai/Metalhead.jl/dev/api/reference/#API-Reference)

In [5]:
vgg = VGG(16; pretrain=true)

LoadError: ArgumentError: Tried to load Base.OneTo(0) into (:pad, :k, :stride) but the structures do not match.

In [6]:
using DataAugmentation

In [7]:
tfm = CenterCrop((224, 224)) |> ImageToTensor()

Sequence{Tuple{Crop{2, DataAugmentation.FromCenter}, ImageToTensor{Float32}}}((Crop{2, DataAugmentation.FromCenter}((224, 224), DataAugmentation.FromCenter()), ImageToTensor{Float32}()))

In [8]:
using Flux
using Flux: onecold, onehotbatch

In [9]:
model = Chain(
    vgg[1:end-1],
    vgg[end][1:end-1],
    # Replace the last layer
    Dense(4096, 256),
    Dense(256, 10)
    ) |> gpu

LoadError: UndefVarError: `vgg` not defined

In [10]:
using MLDatasets

Load the MNIST dataset

In [11]:
function get_data(split)
    data = MNIST(split)
    imgs, y = data.features ./ 255, onehotbatch(data.targets, 0:9);
    X = []
    for i in 1:length(y)
        img = apply(tfm, Image(RGB.(imgs[:,:,i]))) |> itemdata
        push!(X, img)
    end
    loader = Flux.Data.DataLoader((X, y); batchsize=64, shuffle=true) |> gpu;
    return loader
end

get_data (generic function with 1 method)

In [12]:
using Images

In [ ]:
train_loader = get_data(:train);
test_loader = get_data(:test);

**Define a loss function and an optimizer**

In [ ]:
loss_fn = Flux.logitcrossentropy
opt_state = Flux.setup(Adam(3e-3), model[end]) # Freeze the weights of the pre-trained layers

In [ ]:
using ProgressMeter

In [ ]:
epochs = 3

Fine-tune the model

In [ ]:
for epoch in 1:epochs
    @showprogress for (X, y) in train_loader
        # Compute the gradient of the loss with respect to the model's parameters
        loss, ∇ = Flux.withgradient(model) do m
            ŷ = m(X)
            loss_fn(ŷ, y)
        end
        # Update the model's parameters using the optimizer
        Flux.update!(opt_state, model, ∇[1])
    end
    @info "Calculate the accuracy on the test set"
    for (X, y) in test_loader
        ys = onecold(y, 0:9) |> cpu
        ŷ  = onecold(model(X), 0:9) |> cpu
        accuracy = sum( ys .== ŷ ) / length(ys)
        println("Epoch: $epoch, Accuracy: $accuracy")
    end
end